#CSE592 Social Networks Project : LINK PREDICTION IN SOCIAL NETWORKS

#### Imports

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
import networkx as nx
import operator
import random
import math
import scipy
%matplotlib inline
from collections import defaultdict

####Original Data

In [2]:
train_DF = pd.read_csv("newdata.csv", delimiter = "\s")
train_DF.columns = ["From_node", "To_node"]

#### Splitting Data in Training and Test : We have divided the original data by taking 95% training and 5% test data

In [3]:
def get_splitDF():
        global train_DF
        global trainDF
        global testDF
        DF_temp = pd.DataFrame
        rand_nos = np.random.rand(len(train_DF)) < 0.95
        trainDF = train_DF[rand_nos]
        testDF = train_DF[~rand_nos]

#         rand_nos = np.random.rand(len(DF_temp)) < 0.6
#         testDF = DF_temp[rand_nos]
#         evalDF = DF_temp[~rand_nos]

        print 'Train(len) : {0} rows'.format(str(len(trainDF)))
        print 'Test(len) : {0} rows'.format(str(len(testDF)))
        #print 'Eval(len) : {0} rows'.format(str(len(evalDF)))

get_splitDF()

Train(len) : 171454 rows
Test(len) : 9038 rows


#### Creating a networkx graph

In [14]:
def create_graph():
    gph = []
    for i in range(len(trainDF)):
        node1 = trainDF.values[i][0]
        node2 = trainDF.values[i][1]
        gph.append([node1,node2])
    return gph        

In [15]:
#General function to draw graph
def draw_graph(graph, labels=None, graph_layout='shell',
               node_size=5000, node_color='red', node_alpha=0.5,
               node_text_size=30,
               edge_color='blue', edge_alpha=0.3, edge_tickness=0.4,
               edge_text_pos=0.3,
               text_font='sans-serif'):
    fig = plt.figure(figsize=(100,50))
    G=nx.Graph()
    for edge in graph:
        G.add_edge(edge[0], edge[1])
    if graph_layout == 'spectral':
        graph_pos=nx.spectral_layout(G)
    else :
        graph_pos=nx.random_layout(G)
    nx.draw_networkx_nodes(G,graph_pos,node_size=node_size, 
                               alpha=node_alpha, node_color=node_color)
    nx.draw_networkx_edges(G,graph_pos,width=edge_tickness,
                               alpha=edge_alpha,edge_color=edge_color)
    if not graph_layout == 'spectral':
        nx.draw_networkx_labels(G, graph_pos,font_size=node_text_size,
                            font_family=text_font)
    plt.show()

In [16]:
ego_network = create_graph()
G=nx.DiGraph()
#print ego_network
for edge in ego_network:
    #print edge[0], edge[1]
    G.add_edge(edge[0], edge[1])

##Personalized PageRank

#### Computing personalized PageRank from a single source node to all the other nodes and ordering them by their pageranks. The top 50 friends are displayed.

In [ ]:
def pagerank_csv(node,prediction_count):
    personalize1 = {}
    for n in G:
        if n == node:
            personalize1[n] = 1
        else:
            personalize1[n] = 0
    #print personalize
    p1 = nx.pagerank(G, alpha=0.9, personalization=personalize1)
    sorted_x1 = sorted(p1.items(), key=operator.itemgetter(1),reverse = True)
    #print sorted_x
    count =0
    #print G[1].keys()
    lst_pred = []
    for x in sorted_x1:
        if count >= prediction_count:
            break
        else:
            if x[0] not in G[node].keys() and x[0] != node:
                #print x[0]
                lst_pred.append(x[0])
                count +=1
    return lst_pred

###Predicting the top 1000 friends for the 1st 100 users 

In [56]:
pred_dict = {}
count = 0
prediction_count = 1000 #Can be changed according to the number of predictions needed.
for node in G:
    if count>100:
        break
    else:
        preds = pagerank_csv(node,prediction_count)
        pred_dict[node] = preds
        #print node,preds
        count+=1
#print pred_dict

###Getting the test data and converting it to dict where key = source node and value = list of destination nodes for users till user id = 100

In [49]:
test_dict = defaultdict(list)
count = testDF.values[0][0]
for i in range(len(testDF.values)):
    if testDF.values[i][0] < 100:
        #print count,testDF.values[i][1]
        test_dict[testDF.values[i][0]].append(testDF.values[i][1])
    else:
        break
#print dict_list

##Calculating the accuracy of the predictions with respect to the test data

In [52]:
def pred_accuracy():
    count = 0 
    total_count = 0
    for i in test_dict:
        test_l = test_dict[i]
        pred_l = pred_dict[i]
        for j in test_l:
            if j in pred_l:
                count+=1
            total_count+=1
    print "Correct Predictions : " + str(count)
    print "Total count of friends in test data : " + str(total_count)
    accuracy = count/float(total_count)*100
    print "Accuracy for " + str(prediction_count) + "predictions :"accuracy

In [53]:
pred_accuracy()

215 591
36.3790186125
